<a href="https://colab.research.google.com/github/arjunkpreman/Transfer-Learning-Project/blob/main/ResNet50V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Load pre-trained ResNet50V2 model
base_model = ResNet50V2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

94668760/94668760 [==============================] - 1s 0us/step


In [4]:
# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False


In [5]:
# Add new classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)  # num_classes is the number of classes in your task

In [6]:

# Create the transfer learning model
model = Model(inputs=base_model.input, outputs=predictions)

In [7]:

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
# Set up data generators for training and validation
train_datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.resnet_v2.preprocess_input)
train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Dataset/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 3015 images belonging to 10 classes.


In [9]:
val_datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.resnet_v2.preprocess_input)
val_generator = val_datagen.flow_from_directory(
    '/content/drive/MyDrive/Dataset/test',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

Found 921 images belonging to 10 classes.


In [10]:
# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // 32,
    validation_data=val_generator,
    validation_steps=val_generator.samples // 32,
    epochs=5
)

Epoch 1/5
94/94 [==============================] - 931s 10s/step - loss: 0.8660 - accuracy: 0.7177 - val_loss: 0.5124 - val_accuracy: 0.8214
Epoch 2/5
94/94 [==============================] - 17s 176ms/step - loss: 0.2819 - accuracy: 0.9125 - val_loss: 0.4129 - val_accuracy: 0.8538
Epoch 3/5
94/94 [==============================] - 17s 180ms/step - loss: 0.1547 - accuracy: 0.9608 - val_loss: 0.4882 - val_accuracy: 0.8259
Epoch 4/5
94/94 [==============================] - 17s 176ms/step - loss: 0.0962 - accuracy: 0.9789 - val_loss: 0.3282 - val_accuracy: 0.8884
Epoch 5/5
94/94 [==============================] - 17s 176ms/step - loss: 0.0582 - accuracy: 0.9909 - val_loss: 0.3171 - val_accuracy: 0.8996


In [11]:
# Save the trained model
model.save('/content/drive/MyDrive/Dataset/ResNet50V2.h5')

In [12]:

model.evaluate(val_generator)

29/29 [==============================] - 6s 202ms/step - loss: 0.3111 - accuracy: 0.9012


[0.3111063539981842, 0.9011943340301514]